# Spotify Music Recommender via LangChain Agents

In this example we take a look at how we can utilize LangChain Agents to work with the Spotify API to generate music recommendations based off of favorite artists and number of tracks requested.

## Spotify Client Setup
To work with Spotify we will use the Python API: https://developer.spotify.com/documentation/web-api. Full setup instructions can be found in the URL, but ensure you create a project and can access your Client ID and Secret.

<b>Spotipy Docs (Spotify Python Library)</b>: https://spotipy.readthedocs.io/en/2.22.1/

In [ ]:
#!pip install spotipy langchain

In [ ]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import random

client_id = 'Enter here'
client_secret = 'Enter here'

# instantitate spotipy client
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=client_id,
                                                                             client_secret=client_secret))

## LangChain Agents Setup

LangChain Agents make it easy to work with external APIs such as Spotipy. We use a ReAct Agent for our reasoning workflow. LangChain comes with many Tools that are built-in, at the moment of this example there is no built-in tool for the Spotify API, so we build our own custom tool that we then give our Agent access to. This will enable the Agent to take the appropriate actions based off of the input.

### Custom Spotify Tool Creation

In [ ]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from typing import Optional, Type
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import random

client_id = 'defb938e0c81453cb16b1a3219e57aec'
client_secret = '90f029e232eb4411825e7bea1212d69f'

# instantitate spotipy client
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=client_id,
                                                                             client_secret=client_secret))


class MusicInput(BaseModel):
    artists: list = Field(description="A list of artists that they'd like to see music from")
    tracks: int = Field(description="The number of tracks/songs they want returned.")

class SpotifyTool(BaseTool):
    name = "Spotify Music Recommender"
    description = "Use this tool when asked music recommendations."
    args_schema: Type[BaseModel] = MusicInput
    
    # utils
    @staticmethod
    def retrieve_id(artist_name: str) -> str:
        results = sp.search(q='artist:' + artist_name, type='artist')
        if len(results) > 0:
            artist_id = results['artists']['items'][0]['id']
        else:
            raise ValueError(f"No artists found with this name: {artist_name}")
        return artist_id

    @staticmethod
    def retrieve_tracks(artist_id: str, num_tracks: int) -> list:
        if num_tracks > 10:
            raise ValueError("Can only provide up to 10 tracks per artist")
        tracks = []
        top_tracks = sp.artist_top_tracks(artist_id)
        for track in top_tracks['tracks'][:num_tracks]:
            tracks.append(track['name'])
        return tracks

    @staticmethod
    def all_top_tracks(artist_array: list) -> list:
        complete_track_arr = []
        for artist in artist_array:
            artist_id = SpotifyTool.retrieve_id(artist)
            all_tracks = {artist: SpotifyTool.retrieve_tracks(artist_id, 10)}
            complete_track_arr.append(all_tracks)
        return complete_track_arr

    # main execution
    def _run(self, artists: list, tracks: int) -> list:
        num_artists = len(artists)
        max_tracks = num_artists * 10
        print("---------------")
        print(artists)
        print(type(artists))
        print("---------------")
        all_tracks_map = SpotifyTool.all_top_tracks(artists) # map for artists with top 10 tracks
        all_tracks = [track for artist_map in all_tracks_map for artist, tracks in artist_map.items() for track in tracks] #complete list of tracks

        if tracks > max_tracks:
            raise ValueError(f"Only 10 tracks per artist, max tracks for this many artists is: {max_tracks}")
        final_tracks = random.sample(all_tracks, tracks)
        return final_tracks

    def _arun(self):
        raise NotImplementedError("Spotify Music Recommender does not support ")
        
tools = [SpotifyTool()]

### LLM Setup
We need an LLM that's the brains behind our Agent, in this case we specify Bedrock Claude.

In [ ]:
from langchain.llms import Bedrock
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
model_id = "anthropic.claude-v2:1"
model_params = {"max_tokens_to_sample": 500,
                "top_k": 100,
                "top_p": .95,
                "temperature": .5}
llm = Bedrock(
    model_id=model_id,
    model_kwargs=model_params
)

In [ ]:
# sample Bedrock Inference
llm("What is the capitol of the United States?")

## Create Agent & Sample Inference

Here we wrap our tools, llm, and specify our Agent Type. Note that you can make this cleaner with a prompt template/structure, but for demo purposes, we plug in our input text directly in a way the model expects.

In [ ]:
agent = initialize_agent(tools, llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose = True)

In [ ]:
print(agent.run("""I like the following artists: [Arijit Singh, Future, The Weekend], 
                can I get 12 song recommendations with them in it."""))

In [ ]:
# this should error out, because max 30 songs for three artists
print(agent.run("""I like the following artists: [Arijit Singh, Future, The Weekend], 
                can I get 48 song recommendations with them in it."""))